In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#### We Start by importing all of the necessary machine learning libraries

In [4]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from IPython.display import display
from sklearn import metrics

#### Set the path to where your data is to access is it, In my case my data is in a folder called "Data"

In [6]:
PATH = "Data/"

df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates = ["saledate"])

Its  always important to look at your data to make sure you understand the format,how it is stored , what type of variables it holds etc.

In [7]:
def display_all(df):
    with pd.option_context("display.max_rows",1000):
        with pd.option_context("display.max_columns",1000):
            display(df)

In [24]:
display_all(df_raw.tail().transpose())

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


In this project, the evaluation metric is the Root mean Squared Log Error(RMSLE) between the actual and predicted auction prices. The aim is to get the least number

In [8]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In the above line, we have replaced the sale prices with their natural logarithim(their logs).

In [9]:
df_raw.SalePrice

0         11.097410
1         10.950807
2          9.210340
3         10.558414
4          9.305651
5         10.184900
6          9.952278
7         10.203592
8          9.975808
9         11.082143
10        10.085809
11        10.021271
12        10.491274
13        10.325482
14        10.239960
15         9.852194
16         9.510445
17         9.159047
18         9.433484
19         9.350102
20        10.621327
21        10.448715
22        10.165852
23        11.198215
24        10.404263
25         9.433484
26         9.648595
27        10.878047
28        10.736397
29        11.396392
            ...    
401095     9.259131
401096     9.210340
401097     9.047821
401098     9.259131
401099     9.305651
401100     9.259131
401101     9.210340
401102     9.259131
401103     9.433484
401104     9.259131
401105     9.210340
401106     9.259131
401107     9.433484
401108     9.259131
401109     9.259131
401110     9.259131
401111     9.210340
401112     9.259131
401113     9.259131


When you have a date, you want to feature engineer to get a bunch of other columns that could be useful

In [10]:
add_datepart(df_raw, 'saledate')
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [ ]:
We now need to take care of categorical variables that are mostly strings

In [13]:
train_cats(df_raw)

In [30]:
df_raw.UsageBand.cat.set_categories(['High','Medium','Low'], ordered=True, inplace= True)

We still are not quite done. We have missing values which we cant pass as values

In [31]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

At this point we want to save where we are at in a format called feather format. Saves to disk in the same basic format as is in the ram

In [32]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/raw')

We now replace categories with numeric codes,handle missing continuous values and split the dependent variable into a seperate variable

In [37]:
df, y, nas = proc_df(df_raw, 'SalePrice')

Now we go ahead and create a random forest.
We are using a regressor as we are trying to predict a continuous variable which is in our case is sale price.

In [36]:
m = RandomForestRegressor(n_jobs = 1)
m.fit(df,y)
m.score(df,y)

/home/ec2-user/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9830995890168901